# Data Wrangling

This notebook will contain all of the code used to clean the Airbnb listings data within a pandas dataframe and engineer new features for the purpose of building a regression to predict the prices of Airbnb properties in London.

In [1]:
# importing the libraries/modules that will be used within this notebook

import pandas as pd
import geopy.distance
from geopy.extra.rate_limiter import RateLimiter
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# opening the listings.csv document as a pandas dataframe

df = pd.read_csv("../data/listings.csv")

In [3]:
# dataframe has 74 columns and 74840 rows

df.shape

(74840, 74)

## Data Cleaning

Directly below is an alphabetical list of the 74 columns that exist within the Airbnb listings dataset. In the following code, I'm going to decide which columns will be useful for this project, discard the columns that are irrelevant or duplicates and "clean" the columns that, in their current format (e.g. the price column has a dollar sign at the beginning of each value and is thus stored as a piece of text rather than a number), can't be used for modelling purposes. 

In [4]:
sorted(list(df.columns))

['accommodates',
 'amenities',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'calendar_last_scraped',
 'calendar_updated',
 'description',
 'first_review',
 'has_availability',
 'host_about',
 'host_acceptance_rate',
 'host_has_profile_pic',
 'host_id',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_location',
 'host_name',
 'host_neighbourhood',
 'host_picture_url',
 'host_response_rate',
 'host_response_time',
 'host_since',
 'host_thumbnail_url',
 'host_total_listings_count',
 'host_url',
 'host_verifications',
 'id',
 'instant_bookable',
 'last_review',
 'last_scraped',
 'latitude',
 'license',
 'listing_url',
 'longitude',
 'maximum_maximum_nights',
 'maximum_minimum_nights',
 'maxi

## Discarding columns

The first thing I'm going to do is discard the columns that I can immediately see won't be useful for modelling. 

The purpose of this project is to build a model to predict the prices of Airbnb properties that could be used by hosts when they list a new property. Since new properties aren't going to have any existing review data, I'm not going to be able use any review columns within the dataset to help predict their prices. All of the columns below are going to be discarded.

In [6]:
review_cols_drop = ['number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
                    'first_review', 'last_review', 'review_scores_rating',
                    'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                    'review_scores_communication', 'review_scores_location',
                    'review_scores_value', 'reviews_per_month']

df.drop(review_cols_drop,axis=1,inplace=True)

Here are the columns related to the property host that I'm going to be dropping. Some of them are objectively useless as predictor variables (e.g. host_url, host_picture_url) and others are columns that I believe aren't useful (e.g. the name of the host or where they are based).

In [7]:
host_cols_drop = ['host_since', 'host_url', 'host_verifications', 'host_name', 'host_location',
                  'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
                  'host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count_entire_homes',
                  'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms',]

df.drop(host_cols_drop,axis=1,inplace=True)

Below is a mish-mash of columns that either contain no values, are duplicates of other columns or, in my opinion, won't be useful predictor variables.

In [8]:
useless_cols_drop = ['scrape_id', 'last_scraped', 'picture_url', 'neighbourhood',
                     'neighbourhood_group_cleansed', 'bathrooms', 'minimum_nights',
                     'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
                     'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm',
                     'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
                     'availability_30', 'availability_60', 'availability_90', 'availability_365',
                     'calendar_last_scraped', 'license']

df.drop(useless_cols_drop,axis=1,inplace=True)

Before I start any data cleaning, I'm going to rename a couple of the columns so that their titles make more sense.

In [9]:
df.rename(columns={'bathrooms_text': 'bathrooms', 'neighbourhood_cleansed': 'neighbourhood'},inplace=True)

## Filling null values

Let's have a look at the remaining columns that have null values.

In [10]:
null_values = pd.DataFrame(df.isnull().sum())
null_values = null_values[null_values[0] != 0]
null_values

,0
name,23
description,3126
neighborhood_overview,29439
host_about,34283
host_response_time,41905
host_response_rate,41905
host_acceptance_rate,39343
host_is_superhost,1981
host_has_profile_pic,1981
host_identity_verified,1981


### Text Columns

For the text columns containing null values, I'm going to fill them in with the word 'null' to avoid having to remove these rows from my dataset. Later on this project, I'll be able to process this dataset so that these words are ignored by my model. 

In [11]:
# filling the null values in the following columns rather than removing any rows

df.name.fillna('null',inplace=True)
df.description.fillna('null',inplace=True)
df.neighborhood_overview.fillna('null',inplace=True)
df.host_about.fillna('null',inplace=True)

### Beds and bedrooms columns

To begin with, I'm going to deal with the bedrooms column as it contains more null values than the beds column. 

The assumption I'm making is that in the majority of the properties, the number of beds will match the number of bedrooms. As such, where the beds value is not null or 0, I'm going to use it to fill in the corresponding null bedrooms value. Where the beds value is null or zero, I'm going to fill it in with the median bedrooms value, which is 1, instead.

In [12]:
# filling in the null values in the bedrooms with the beds value, as these will generally be a straight match

def bedroom_cleaner(row):
    if np.isnan(row['bedrooms']) and row['beds'] != 0 and not np.isnan(row['beds']):
        return row['beds']
    else:
        return df.bedrooms.median()
    
df['bedrooms'] = df.apply(bedroom_cleaner,axis=1)

For the beds column, I'm just going to fill the null values with the column's median value of 1.

In [13]:
# filling null values in the beds column with the median values

df.beds.fillna(df.beds.median(),inplace=True)

I've created a function to allow me to look at the distribution of values in each variable

In [ ]:
# # function for analysing a variable

# def variable_viewer(x):
#     values = df[x].value_counts(sort=False)
#     proportion = df[x].value_counts(sort=False,normalize='all')
#     variable_df = pd.DataFrame({'value_counts': values, 'proportion': proportion})
#     return variable_df.sort_values('value_counts', ascending=False)

### Converting true/false columns to binary values

For the columns containing 't' (true), 'f' (false) and null values. I'm going to convert them to contain binary values, where 1 represents true and 0 is false or null.

In [14]:
columns_to_binarise = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']

In [15]:
for column in columns_to_binarise:
    df[column] = df[column].apply(lambda x: 1 if x=='t' else 0)

### Host response time

A larger number of the host response time values are null values. So we can one-hot encode this column for modelling, I'm going to convert the null values to 'unknown'.

In [16]:
df.host_response_time.value_counts(normalize='all',dropna=False)

NaN                   0.559928
within an hour        0.224506
within a few hours    0.093560
within a day          0.076951
a few days or more    0.045056
Name: host_response_time, dtype: float64

In [17]:
df.host_response_time.fillna('unknown',inplace=True)

### Host acceptance rate

Like previously, a lot of the values in the host acceptance rate column are null. This causes an issue as there are too many null values to fill with an aggregated value, but if we don't fill them, then we aren't able to use the non-null acceptance rates as they are.

The best solution in this instance is to categorize the values before one-hot encoding them. The Airbnb host guide (https://blog.atairbnb.com/hospitality-starts-with-accepting-reservations/) states that the top hosts have an acceptance rate of 75% or above, so I'll categorize the rates in my dataset depending on whether they're in this bracket or if they either aren't or are null. 

In [23]:
df.host_acceptance_rate.value_counts(normalize='all', dropna=False).head(10)

NaN     0.525695
100%    0.166635
0%      0.042718
96%     0.020604
50%     0.018520
98%     0.015794
95%     0.015032
94%     0.011130
67%     0.011077
99%     0.010115
Name: host_acceptance_rate, dtype: float64

In [24]:
df['host_acceptance_rate'] = df.host_acceptance_rate.str.replace('%','').fillna(np.nan).astype('float')

In [25]:
# function to categorize host acceptance rates

def host_filler(x):
    if not np.isnan(x) and x >= 75:
        return "Above or equal to 75%"
    else:
        return "Below 75% or not displayed"

In [26]:
df['host_acceptance_rate'] = df.host_acceptance_rate.apply(host_filler)

### Host response rate

Since the host response rate column has a similar issue as the host acceptance rate, it makes sense to solve it using the same method.

In [28]:
df.host_response_rate.value_counts(normalize='all', dropna=False).head(10)

NaN     0.559928
100%    0.258739
0%      0.029329
90%     0.014645
97%     0.013709
98%     0.011317
50%     0.011184
95%     0.009514
80%     0.009460
67%     0.008685
Name: host_response_rate, dtype: float64

In [29]:
# removing the % sign, filling null values with np.nan and converting the values to floats

df['host_response_rate'] = df.host_response_rate.str.replace('%','').fillna(np.nan).astype('float')

In [30]:
df['host_response_rate'] = df.host_response_rate.apply(host_filler)

### Re-checking for null values

No more columns with null values!

In [31]:
null_values = pd.DataFrame(df.isnull().sum())
null_values = null_values[null_values[0] != 0]
null_values

,0


## Dealing with columns on an individual basis

### Bathrooms column

As the example below shows, the column showing the number of bathrooms in each property is stored as a text value. The code I've written beneath the example will convert the bathroom value from text to a float.

In [ ]:
# INSERT EXAMPLE WHAT THE BATHROOMS TEXT COLUMN LOOKS LIKE

df.bathrooms[0]

In [ ]:
# function to check if string value is numeric

def is_number(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

# function to convert bathrooms_text values

def bathroom_cleaner(x):
    try:
        split = x.lower().split()
        if is_number(split[0]):
            return float(split[0])
        elif 'half-bath' in split:
            return float(0.5)
        else:
            return float(x)
    except:
        return x
    
# replacing old bathrooms_text variable

df['bathrooms_text'] = df.bathrooms_text.apply(bathroom_cleaner)

### Cleaning the property_type column

At the moment the property type column contains too many variables, some with very few values. I'm hoping that a model will perform better if these values are combined in to umbrella categories instead.

In [ ]:
df.property_type.value_counts().head(50)

In [ ]:
# new property type categories

property_categories = ['apartment', 'house', 'townhouse', 'condominium',
                       'hotel', 'boutique hotel', 'bed and breakfast', 'loft',
                       'guest suite', 'guesthouse', 'private room', 'aparthotel',
                      'bungalow', 'hostel', 'boat', 'cottage', 'bungalow', 'villa', 'houseboat', 'other']

# function to sort the property column in to new categories

def property_simplifier(x):
    split = x.lower().split()
    if (' ').join(split[-3:]) in property_categories:
        return (' ').join(split[-3:])
    elif (' ').join(split[-2:]) in property_categories:
        return (' ').join(split[-2:])
    elif split[-1]=='houseboat':
        return 'boat'
    elif split[-1] in property_categories:
        return split[-1]
    else:
        return 'other'
    
# apply function to property_type column

df['property_type_simplified'] = df.property_type.apply(property_simplifier)

In [ ]:
apartment_cats = ['apartment', 'condominium', 'loft', 'guest suite', 'private room',
                  'hotel', 'boutique hotel', 'bed and breakfast', 'guest suite',
                  'aparthotel', 'hostel']

df['property_type_basic'] = df.property_type_simplified.apply(lambda x: "apartment" if x in apartment_cats else "house")

In [ ]:
df.property_type_basic.value_counts()

### Dummifying amenities column

Discarding the host_verification values - not important

In [ ]:
# converting the list values in the amenities column in to binary dummified columns

amenities_values = []

for amenities_list in df.amenities:
    if eval(amenities_list) != None:
        lst = eval(amenities_list)
        for value in lst:
            if value not in amenities_values:
                amenities_values.append(value)
            
amenities_dict = {}

for value in amenities_values:
    amenities_dict[value] = []    
    
for amenities_list in df.amenities:
    if eval(amenities_list) != None:
        lst = eval(amenities_list)
        for key in amenities_dict.keys():
            if key in lst:
                amenities_dict[key].append(1)
            else:
                amenities_dict[key].append(0)
    else:
        for key in amenities_dict.keys():
            amenities_dict[key].append(0)

In [ ]:
amenities_count_dict = {}

for key in amenities_dict.keys():
    amenities_count_dict[key] = sum(amenities_dict[key])

In [ ]:
over_500 = {}

for item in amenities_count_dict.keys():
    if amenities_count_dict[item] >= 500:
        over_500[item] = amenities_count_dict[item]

sorted(over_500.items())

In [ ]:
amenities_cats_dict = {'air_conditioning': 'air conditioning', 
                       'bbq': 'bbq',
                       'baby_facilities': 'baby|crib|changing table|high chair', 
                       'balcony_or_patio': 'patio|balcony',
                       'bath': 'bathtub|bath', 
                       'bed_linen': 'bed linens', 
                       'cable_tv': 'cable',
                       'child_friendly': 'children', 
                       'coffee_maker': 'coffee|nespresso', 
                       'cooking_facilities': 'oven|stove',
                       'dishwasher': 'dishwasher', 
                       'fridge_freezer': 'refrigerator|fridge|freezer',
                       'garden': 'backyard|garden', 
                       'has_workspace': 'workspace', 
                       'host_greets_you': 'host greets you',
                       'long_term_stays': 'long term stays allowed',
                       'luggage_dropoff': 'luggage dropoff', 
                       'lock_on_bedroom_door': 'lock on bedroom',
                       'luxury_facilities': 'gym|hot tub|pool|sauna', 
                       'private_entrance': 'private entrance',
                       'toiletries': 'soap|conditioner|shampoo|shower gel', 
                       'tumble_dryer': 'Dryer',
                       'tv': 'tv'}

for category in amenities_cats_dict.keys():
    if category == 'tumble_dryer':
        df.loc[df['amenities'].str.contains(amenities_cats_dict[category], case = True), category] = 1
        df.loc[~df['amenities'].str.contains(amenities_cats_dict[category], case = True), category] = 0
    else:    
        df.loc[df['amenities'].str.contains(amenities_cats_dict[category], case = False), category] = 1
        df.loc[~df['amenities'].str.contains(amenities_cats_dict[category], case = False), category] = 0

### Removing dollar sign from the price variable

In [ ]:
df.price.str.replace('$','').str.replace('.00','').str.replace(',','').replace('',0).astype('float')

Removing the properties with zero value for price from the dataframe. From looking at the Airbnb listings, these seem to be properties with zero availability. This is likely why Inside Airbnb were unable to scrape the data for them.

In [ ]:
# # Removing the properties with a zero value for price

# price_0 = df[df.price==0]

# price_0.shape[0]

In [ ]:
# df.drop(price_0.index,axis=0,inplace=True)

# HERE IS WHERE TO ADDRESS AND RESOLVE THE ISSUES WITH THE PRICE VALUES

# Feature Engineering

### Creating new distance features using the long and lat variables

#### Distance from "centre" of London

Making an assertion that trafalgar square is the centre of London.

In [ ]:
trafalgar_square = (51.504831314, -0.123499506)

In [ ]:
df['distance_from_center'] = df.apply(lambda row: geopy.distance.distance((row['latitude'],row['longitude']),trafalgar_square).km,axis=1)

#### Nearest train station

In [ ]:
stations = pd.read_csv('../data/Stations_20180921.csv')

Code to calculate the closest train station to each property and how far away it is in km.

In [ ]:
# def station_checker(lat,long):
#     station = ''
#     station_distance = 1000
#     for station_,lat_, long_ in zip(stations.NAME,stations.y,stations.x):
#         calculated_distance = geopy.distance.distance((lat,long),(lat_,long_)).km
#         if calculated_distance < station_distance:
#             station_distance = calculated_distance
#             station = station_
#     return station, station_distance

Storing the results in a dictionary

In [ ]:
# station_dict = {'index': [], 'nearest_station': [], 'station_distance': []}


# for i in df.index:
#     station_checker_result = station_checker(df.loc[i]['latitude'],df.loc[i]['longitude'])
#     station_dict['index'].append(i)
#     station_dict['nearest_station'].append(station_checker_result[0])
#     station_dict['station_distance'].append(station_checker_result[1])

# station_df = pd.DataFrame(station_dict)
# station_df.to_csv('../data/station_df.csv')

In [ ]:
# creating a dataframe out of the station data

station_df = pd.read_csv('../data/station_df.csv',index_col=1)

station_df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
# adding the series to our dataframe

df['nearest_station'] = station_df.nearest_station
df['station_distance'] = station_df.station_distance

### Adding average rental price for the area

data sourced from https://www.ons.gov.uk/peoplepopulationandcommunity/housing/adhocs/12871privaterentalmarketinlondonjanuarytodecember2020

In [ ]:
locator = geopy.geocoders.Nominatim(user_agent='myGeocoder',timeout=10)

rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

Code to find out the postcode of the property using the co-ordinates

In [ ]:
# def postcode_generator(row):
#     co_ordinates = (row['y'],row['x'])
#     try:
#         location = rgeocode(co_ordinates)
#         postcode = location.raw['address']['postcode'].split()[0]
#         return postcode
#     except:
#         return "error"

# stations['postcode'] = stations.apply(postcode_generator,axis=1)

# stations.to_csv('../data/stations_with_postcode.csv')

In [ ]:
stations = pd.read_csv('../data/stations_with_postcode.csv',index_col=0)

In [ ]:
# preparing the stations dataframe for the join

stations.rename(columns={'NAME': 'nearest_station'},inplace=True)

stations.set_index('nearest_station',inplace=True)

In [ ]:
# joining the stations data with our dataframe

df = df.join(stations, on='nearest_station', how='left')

# removing the columns we don't need

df.drop(['FID','OBJECTID','EASTING','NORTHING','x','y', 'LINES'],axis=1,inplace=True)

df.rename({'NETWORK':'rail_network','Zone':'tfl_zone'},axis=1,inplace=True)

In [ ]:
# preparing the rental prices dataframe for the join

rental_prices = pd.read_csv('../data/londonrentalstatisticsq42020.csv')
rental_prices['Mean'] = rental_prices.Mean.apply(lambda x: float(x.replace(',','')))

rental_prices.set_index('Postcode District',inplace=True)
rental_prices.drop('Bedroom Category',axis=1,inplace=True)
rental_prices.rename(columns={'Mean': 'mean_monthly_rent'},inplace=True)

In [ ]:
df = df.join(rental_prices,on='postcode',how='left')

Filling the null values in the mean rent column

In [ ]:
def rent_filler(row):
    if np.isnan(row['mean_monthly_rent']):
        mean_neighbourhood_rent = df[df.neighbourhood_cleansed==row['neighbourhood_cleansed']]['mean_monthly_rent'].mean()
        return mean_neighbourhood_rent
    else:
        return row['mean_monthly_rent']

In [ ]:
df['mean_monthly_rent'] = df.apply(rent_filler,axis=1)

### Adding serviced variable

In [ ]:
def serviced_function(x):
    serviced_prop_types_list = ['serviced', 'hotel', 'bed and breakfast', 'aparthotel', 'hostel']
    if any([prop_type in x for prop_type in serviced_prop_types_list]):
        return 1
    else:
        return 0
    
df['serviced_property'] = df.property_type.apply(serviced_function)

In [ ]:
# creating new columns to show whether the properties have text variables such as description, host_about etc.

df['description_provided'] = df.description.apply(lambda x: 0 if x == 'null' else 1)
df['neighborhood_overview_provided'] = df.neighborhood_overview.apply(lambda x: 0 if x == 'null' else 1)
df['host_about_provided'] = df.host_about.apply(lambda x: 0 if x == 'null' else 1)

In [ ]:
null_values = pd.DataFrame(df.isnull().sum())
null_values = null_values[null_values[0] != 0]
null_values

### Creating new variables to show length of text data

In [ ]:
nlp_columns = ['name', 'description', 'neighborhood_overview', 'host_about']

def text_counter(text):
    if text != 'null':
        split = text.split()
        return len(split)
    else:
        return 0

for column in nlp_columns:
    df[column+'_length'] = df[column].apply(text_counter)

To-Do List

Data Cleaning:

- apply lower and higher limits to the price variable to deal with outliers - include visualisation showing the issue with outliers
- tidy notebook: remove cells that aren't needed, add code comments, write-up markdown cells

Variable Transformation:

- look at distributions of continuous/discrete variables - do they need transforming?
- look in to log transforming the continuous variables (naive-Bayes lessons)

Modelling:

- review the use of NLP - could we instead look at key words within the variables? This might be a better option for the title variable
- can we use neural networks?

good visualisations: https://towardsdatascience.com/predicting-airbnb-prices-with-deep-learning-part-2-how-to-improve-your-nightly-price-50ea8bc2bd29

Change Tracker:

- added serviced column
- removed host_since column